In [ ]:
# !pip install imutils
# !pip install opencv-python

In [2]:
from pymongo import MongoClient
from pathlib import Path
from tqdm.notebook import tqdm
from shutil import copyfile
import imutils
import cv2
from PIL import Image
import json
from itertools import chain
from pydash import pick

from lib.parallel import parallel

In [3]:
images_dir = Path('./images')
gallery_dir = Path('./bad-vis-images')

In [4]:
!rm -r {gallery_dir}

In [5]:
gallery_dir.mkdir(exist_ok=True)
preview_dir = gallery_dir/'preview'
preview_dir.mkdir(exist_ok=True)
thumbnail_dir = gallery_dir/'thumbnail'
thumbnail_dir.mkdir(exist_ok=True)
meta_dir = gallery_dir/'meta'
meta_dir.mkdir(exist_ok=True)
imagelist_dir = gallery_dir/'image_lists'
imagelist_dir.mkdir(exist_ok=True)

In [6]:
mongo = MongoClient('172.17.0.1', 27017)
db = mongo['bad-vis']
posts = db['posts']
imagededup = db['imagededup']
visimages = db['visimages']

In [7]:
visimages.drop()

In [8]:
thumbnail_size = 140

In [9]:
def finalize_image (image):
    image['image_name'] = f"{image['id']}_{image['index_in_album']}"
    image['image_path'] = str(preview_dir/f"{image['image_name']}{image['ext']}")
    copyfile(image['file_path'], image['image_path'])

    image['thumbnail_path'] = str(thumbnail_dir/f"{image['image_name']}.jpg")
    ratio = thumbnail_size / max([image['width'], image['height']])
    image['thumbnail_width'] = min([thumbnail_size, int(round(image['width'] * ratio))])
    image['thumbnail_height'] = min([thumbnail_size, int(round(image['height'] * ratio))])

    im = Image.open(image['file_path']).convert('RGB')
    im.thumbnail((image['thumbnail_width'], image['thumbnail_height']))
    im.save(image['thumbnail_path'], 'JPEG')

    del image['_id']

    with open(meta_dir/f"{image['image_name']}.json", 'w') as f:
        json.dump(image, f)

    visimages.insert_one(image)

    return image

In [10]:
images = parallel(finalize_image, imagededup.find(), total=imagededup.estimated_document_count())

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:2766: DecompressionBombWarning: Image size (135364608 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:2766: DecompressionBombWarning: Image size (100000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


# Image Lists

In [11]:
visImages = [i for i in visimages.find()]

In [12]:
visImages.sort(key=lambda x: x['popularity_score'])

In [13]:
visImages.reverse()

In [14]:
attrs = [
    'id',
    'image_name',
    'image_path',
    'thumbnail_path',

    'datetime',
    'url',
    'title',
    'author',
    'source',
    'source_url',
    'source_platform',
    'popularity_score',

    'size',
    'width',
    'height',
    'phash',

    'animated',
    'tags',
    'labels',
]

In [15]:
for i in range(len(visImages)):
    del visImages[i]['_id']

In [16]:
visImages = [pick(i, *attrs) for i in visImages]

In [17]:
json.dump({'images': visImages}, open(imagelist_dir/'all.json', 'w'), separators=(',', ':'))

In [18]:
batch_cuts = [0] + [i for i in range(225, len(visImages), 400)] + [len(visImages)]
for (batch_begin, batch_end) in zip(batch_cuts[:-1], batch_cuts[1:]):
    json.dump({
        'images': visImages[batch_begin:batch_end],
        'next': f"image_lists/{batch_end}.json" if batch_end != len(visImages) else None
    }, open(imagelist_dir/f"{batch_begin}.json", 'w'), separators=(',', ':'))

# Sprite

In [19]:
# ims = []
# sprites_path = str(gallery_dir/'0.jpg')
# for i in range(225):
#     visImage = visImages[i]
#     visImage['sprites_path'] = sprites_path
#     visImage['sprites_pos_x'] = (i % 15) * thumbnail_size
#     visImage['sprites_pos_y'] = (i // 15) * thumbnail_size
#     im = cv2.imread(visImage['thumbnail_path'])
#     w, h = im.shape[:2]
#     new_im = cv2.copyMakeBorder(im, 0, thumbnail_size-w, 0, thumbnail_size-h, cv2.BORDER_CONSTANT, value=[0, 0, 0])
#     ims.append(new_im)
# montages = imutils.build_montages(ims, (140, 140), (15, 15))
# cv2.imwrite(sprites_path, montages[0])